<img src="images/img.png" />

# CS5228 Project, Group 32

In [35]:
# Auto reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Preprocessing
In this part, we are going to perform some data preprocessing steps. This may include:
* Data cleaning: handle missing values, duplicates, inconsistant or invalid vallues, outliers

* Data reduction: reduce number of attributes, reduce number of attribute values

* Data transformation: attribute construction, normalization

* Data discretization: encode to numerical attributes

### Setting up the Notebook

In [39]:
import os
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [41]:
# Load file into pandas dataframe
df = pd.read_csv('./data/train.csv')

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points in training data, each with 30 attributes.


### Data Cleaning

Before data cleaning, remove the known attributes that are not meaningful to our prediction model:
  * Meaningless idendifier: listing_id 
  * Attributes in free text: title, description, features, accessories
  * Attribute with the same value: eco_category, indicative_price
  * Attribute unlikely to affect price: curb_weight

In [45]:
columns_to_drop = [
    'listing_id',          # Meaningless identifier
    'title',               # Attributes in free text
    'description',
    'features',
    'accessories',
    'eco_category',        # Attribute with the same value
    'indicative_price',
    'curb_weight',         # Attribute unlikely to affect price
    'original_reg_date',
    'lifespan',
]

df = df.drop(columns=columns_to_drop)

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points in training data, each with 20 attributes.


### Handle Missing Values
Firstly, for each of the columns with missing value, check the number of rows with NaN values.
There are 3 scenarios:
1. NaN value is the major (e.g. fuel_type has 19121 rows with NaN values), we remove the corresponding attritubes.
2. NaN value is the minor. We can choose to fill or delete related data points. 

In [48]:
columns_to_check = [
    'make',
    'fuel_type',
    'manufactured',
    'power',
    'engine_cap',
    'mileage',
    'no_of_owners',
    'depreciation',
    'road_tax',
    'dereg_value',
    'omv',
    'arf',
    'opc_scheme'
]

# Calculate the number of NaN values in each specified column
nan_counts = df[columns_to_check].isna().sum()

# Print the number of NaN values for each column
print('Training data')
for column, count in nan_counts.items():
    print(f"Column '{column}' has {count} rows with NaN values.")

Training data
Column 'make' has 1316 rows with NaN values.
Column 'fuel_type' has 19121 rows with NaN values.
Column 'manufactured' has 7 rows with NaN values.
Column 'power' has 2640 rows with NaN values.
Column 'engine_cap' has 596 rows with NaN values.
Column 'mileage' has 5304 rows with NaN values.
Column 'no_of_owners' has 18 rows with NaN values.
Column 'depreciation' has 507 rows with NaN values.
Column 'road_tax' has 2632 rows with NaN values.
Column 'dereg_value' has 220 rows with NaN values.
Column 'omv' has 64 rows with NaN values.
Column 'arf' has 174 rows with NaN values.
Column 'opc_scheme' has 24838 rows with NaN values.


We delete attributes with TOO many NaN value here.

In [51]:
columns_to_drop_nan = [
    'fuel_type',
    'opc_scheme'
]

df = df.drop(columns=columns_to_drop_nan)

Then we try to fill up other missing values.

In [54]:
from util.DataPreprocess import HandlingMissingValues

df = HandlingMissingValues(df)

NaN values after handling:  0


### Remove Exact Duplicates
We remove duplicated data points here.

In [56]:
df = df.drop_duplicates()

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 24258 data points in training data, each with 18 attributes.


### Merge rows with fewer data points on specific attributes

### Transform categorical value to numerical values

In [59]:
categorical_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'transmission',
]

encode_dict = {}
le = LabelEncoder()
for column in categorical_columns:
    df[column] = le.fit_transform(df[column])
    encode_dict[column] = {str(label): int(index) for index, label in enumerate(le.classes_)}

with open('./data/encode.json', 'w') as file:
    json.dump(encode_dict, file, indent=4)

### Transform date time attributes to numerical values

In [61]:
df['reg_date'] = pd.to_datetime(df['reg_date'], format='%d-%b-%Y')
df['reg_year'] = df['reg_date'].dt.year
df = df.drop(columns=['reg_date'])

num_records, num_attributes = df.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 24258 data points, each with 18 attributes.


### Handle category attribute

In [67]:
from util.DataPreprocess import HandlingCategoryAttribute

df = HandlingCategoryAttribute(df)

Number of unique categories: 15
Unique categories: {'parf car', 'opc car', 'electric cars', 'premium ad car', 'imported used vehicle', 'sgcarmart warranty cars', 'coe car', 'consignment car', 'direct owner sale', 'vintage cars', 'hybrid cars', 'sta evaluated car', 'almost new car', 'rare & exotic', 'low mileage car'}
There are 24258 data points, each with 32 attributes.


### Remove outliers

In [70]:
# from util.DataPreprocess import OutlierRemoval

# df = OutlierRemoval(df, 'model', 'price')

### Saving the Data

In [73]:
file_name = './data/train_preprocessed.csv'

# Check if the file exists
if os.path.exists(file_name):
    # Delete the file
    os.remove(file_name)
    print(f"Existing file '{file_name}' has been deleted.")

# Save the DataFrame to CSV
df.to_csv(file_name, index=False)
print(f"DataFrame has been saved to '{file_name}'.")

Existing file './data/train_preprocessed.csv' has been deleted.
DataFrame has been saved to './data/train_preprocessed.csv'.


## Data Mining

### Load preprocessed training data

In [264]:
# Load file into pandas dataframe, we saved our preprocessed file at path 'output_file'
training_file = './data/train_preprocessed.csv'
df = pd.read_csv(training_file)

columns_to_keep = [
    'model',
    'mileage',
    'low mileage car',
    'manufactured',
    'reg_year',
    'type_of_vehicle',
    'dereg_value',
    'depreciation',
    'power',
    'coe',
    'arf',
    'omv',
    'price',
    'road_tax',
    'almost new car',
    'coe car',
    'parf car',
]

df = df[columns_to_keep]
columns_to_keep = [col for col in df.columns if col != 'price']

num_records, num_attributes = df.shape
print("There are {} data points in training data, each with {} attributes.". format(num_records, num_attributes))

There are 24258 data points in training data, each with 17 attributes.


### Load test data and preprocess

In [267]:
test_file = './data/test.csv'
df_test = pd.read_csv(test_file)

df_test['reg_date'] = pd.to_datetime(df_test['reg_date'], format='%d-%b-%Y')
df_test['reg_year'] = df_test['reg_date'].dt.year
df_test = df_test.drop(columns=['reg_date'])

# Replace '-' with an empty string
df_test['category'] = df_test['category'].replace('-', '')

# Split the 'category' column into lists
df_test['category_list'] = df_test['category'].str.split(', ')

# Handle empty strings by replacing them with empty lists
df_test['category_list'] = df_test['category_list'].apply(lambda x: [] if x == [''] else x)

# Import itertools for flattening lists
from itertools import chain

# Flatten the list of lists to a single list
all_categories = list(chain.from_iterable(df_test['category_list']))

# Get the unique categories
unique_categories = set(all_categories)

# Print the number of unique categories
print(f"Number of unique categories: {len(unique_categories)}")
print("Unique categories:", unique_categories)

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the category lists
category_dummies = mlb.fit_transform(df_test['category_list'])

# Create a DataFrame with the one-hot encoded categories
category_df = pd.DataFrame(category_dummies, columns=mlb.classes_, index=df_test.index)

# Concatenate the new dummy columns to the original DataFrame
df_test = pd.concat([df_test, category_df], axis=1)

# Drop the temporary 'category_list' column if desired
df_test.drop('category_list', axis=1, inplace=True)
df_test.drop('category', axis=1, inplace=True)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

Number of unique categories: 15
Unique categories: {'parf car', 'opc car', 'electric cars', 'premium ad car', 'imported used vehicle', 'sgcarmart warranty cars', 'coe car', 'consignment car', 'direct owner sale', 'vintage cars', 'hybrid cars', 'sta evaluated car', 'almost new car', 'rare & exotic', 'low mileage car'}
There are 24258 data points, each with 17 attributes.


### Data Augmentation, copy rows with less than 5 samples by group

In [270]:
from util.DataPreprocess import DataAugmentation

df_aug = DataAugmentation(df)

num_records, num_attributes = df_aug.shape
print("There are {} data points after augmentation, each with {} attributes.". format(num_records, num_attributes))

There are 48306 data points after augmentation, each with 17 attributes.


### Select attributes on test data

In [275]:
num_records, num_attributes = df_test.shape
print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

categorical_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'transmission',
]

with open('./data/encode.json', 'r') as file:
    data = json.load(file)

for col, cate_dict in data.items():
    if col in df_test.columns:
        df_test[col] = df_test[col].map(cate_dict)

df_test = df_test[columns_to_keep]

num_records, num_attributes = df_test.shape
print("There are {} data points in test data, each with {} attributes.". format(num_records, num_attributes))
df_test

There are 10000 data points, each with 43 attributes.
There are 10000 data points in test data, each with 16 attributes.


,model,mileage,low mileage car,manufactured,reg_year,type_of_vehicle,dereg_value,depreciation,power,coe,arf,omv,road_tax,almost new car,coe car,parf car
0,705.0,112000.0,0,2015.0,2015,8,9582.0,17660.0,96.0,57199,9229.0,19229.0,682.0,0,0,1
1,36.0,120000.0,1,2007.0,2007,3,13644.0,10920.0,79.0,42564,15782.0,14347.0,1113.0,0,1,0
2,221.0,43000.0,0,2019.0,2020,6,54818.0,22120.0,141.0,32801,47809.0,39863.0,1210.0,0,0,1
3,707.0,53300.0,0,2019.0,2019,3,26363.0,13700.0,79.0,29159,15573.0,15573.0,682.0,0,0,1
4,36.0,149000.0,0,2015.0,2015,1,15197.0,14190.0,88.0,56001,13097.0,18097.0,682.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,389.0,50000.0,0,2020.0,2020,8,40705.0,16830.0,119.0,41503,19388.0,19388.0,680.0,0,0,1
9996,355.0,8500.0,1,2022.0,2023,8,104529.0,21290.0,96.0,86000,39722.0,34087.0,586.0,0,0,1
9997,46.0,163955.0,0,2010.0,2010,6,22004.0,12060.0,115.0,34935,42233.0,42233.0,1691.0,0,1,0
9998,201.0,85473.0,0,2019.0,2020,3,28242.0,13720.0,93.0,30010,14799.0,14799.0,738.0,0,0,1


In [289]:
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


def rf_fill_missing_values(df_train, df_test, n_neighbors=5):
    combined_df = pd.concat([df_train, df_test], ignore_index=True)
    

    if combined_df['manufactured'].isnull().any():
        
        known_manufactured = combined_df[combined_df['manufactured'].notna()]
        unknown_manufactured = combined_df[combined_df['manufactured'].isna()]
        
      
        rf_clf = RandomForestClassifier()
        rf_clf.fit(known_manufactured.drop(columns=['manufactured']), known_manufactured['manufactured'])
        
        
        combined_df.loc[unknown_manufactured.index, 'manufactured'] = rf_clf.predict(unknown_manufactured.drop(columns=['manufactured']))

    numeric_features = ['mileage', 'dereg_value', 'depreciation', 'power', 'arf', 'omv', 'road_tax']
    
    for feature in numeric_features:
        if combined_df[feature].isnull().any():
            
            known_data = combined_df[combined_df[feature].notna()]
            unknown_data = combined_df[combined_df[feature].isna()]
            
        
            X_train = known_data.drop(columns=numeric_features)
            y_train = known_data[feature]
            rf_model = RandomForestRegressor()
            rf_model.fit(X_train, y_train)
            
            combined_df.loc[unknown_data.index, feature] = rf_model.predict(unknown_data.drop(columns=numeric_features))

    df_test_filled = combined_df.loc[df_train.shape[0]:].reset_index(drop=True)

    return df_test_filled

df_test = rf_fill_missing_values(df_aug, df_test)




In [291]:
null_counts = df_test.isnull().sum()
null_counts = null_counts[null_counts > 0]
for column, count in null_counts.items():
    print(f"{column}: {count} null values")
df_test

model: 98 null values
price: 10000 null values


,model,mileage,low mileage car,manufactured,reg_year,type_of_vehicle,dereg_value,depreciation,power,coe,arf,omv,price,road_tax,almost new car,coe car,parf car
0,705.0,112000.0,0,2015.0,2015,8,9582.0,17660.0,96.0,57199,9229.0,19229.0,NaN,682.0,0,0,1
1,36.0,120000.0,1,2007.0,2007,3,13644.0,10920.0,79.0,42564,15782.0,14347.0,NaN,1113.0,0,1,0
2,221.0,43000.0,0,2019.0,2020,6,54818.0,22120.0,141.0,32801,47809.0,39863.0,NaN,1210.0,0,0,1
3,707.0,53300.0,0,2019.0,2019,3,26363.0,13700.0,79.0,29159,15573.0,15573.0,NaN,682.0,0,0,1
4,36.0,149000.0,0,2015.0,2015,1,15197.0,14190.0,88.0,56001,13097.0,18097.0,NaN,682.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,389.0,50000.0,0,2020.0,2020,8,40705.0,16830.0,119.0,41503,19388.0,19388.0,NaN,680.0,0,0,1
9996,355.0,8500.0,1,2022.0,2023,8,104529.0,21290.0,96.0,86000,39722.0,34087.0,NaN,586.0,0,0,1
9997,46.0,163955.0,0,2010.0,2010,6,22004.0,12060.0,115.0,34935,42233.0,42233.0,NaN,1691.0,0,1,0
9998,201.0,85473.0,0,2019.0,2020,3,28242.0,13720.0,93.0,30010,14799.0,14799.0,NaN,738.0,0,0,1


### Check if train data has all models in test data

In [130]:
models_in_df = set(df['model'].unique())
models_in_df_test = set(df_test['model'].unique())

if models_in_df_test.issubset(models_in_df):
    print("df includes all models in df_test")
else:
    missing_models = models_in_df_test - models_in_df
    print("df does not include", missing_models)

df does not include {nan}


### Mining code here

In [92]:
from util.DataMining import split_dataframe, split_dataframe_flex
from util.DataMining import (
    RandomForestMining,
    RandomForestMiningByModel,
    GradientBoostingMining,
    LinearRegressionMining,
    LinearRegressionMiningByModel,
    CombinedDataMiningRandomForestAndLinearRegression
)

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [94]:
# run_times, rmse_sum = 5, 0
# for i in tqdm(range(run_times), desc='Running Random Forest'):
#     target_col = 'price'
#     x_train, x_test, y_train, y_test = split_dataframe(df, target_col)
#     rmse_sum += RandomForestMining(x_train, x_test, y_train, y_test)
# print('Average RMSE:', round(rmse_sum / run_times))

In [309]:
run_times, rmse_sum = 1, 0
for i in tqdm(range(run_times), desc='Running Random Forest'):
    train_drop_cols = ['price']
    test_cols = ['price']
    x_train, x_test, y_train, y_test = split_dataframe_flex(df_aug, train_drop_cols, test_cols)
    rmse_sum += RandomForestMining(x_train, x_test, y_train, y_test)
print('Average RMSE:', round(rmse_sum / run_times))

Running Random Forest:   0%|                              | 0/1 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Running Random Forest: 100%|██████████████████████| 1/1 [01:16<00:00, 76.19s/it]

Running not in develop mode
RMSE on test data: 9016.710653884204
Average RMSE: 9017


In [317]:
import joblib  
from tqdm import tqdm

run_times, rmse_sum = 10, 0
best_model = None  
best_rmse = float('inf')  

for i in tqdm(range(run_times), desc='Running Random Forest'):
    train_drop_cols = ['price']
    test_cols = ['price', 'model']
    x_train, x_test, y_train, y_test = split_dataframe_flex(df_aug, train_drop_cols, test_cols)
    
    
    rf_model, rmse = RandomForestMiningByModel(x_train, x_test, y_train, y_test)  
    
    rmse_sum += rmse  # 

    
    if rmse < best_rmse:  
        best_rmse = rmse
        best_model = rf_model  
        joblib.dump(rf_model, 'best_random_forest_bymodel.pkl')  

print('Average RMSE:', round(rmse_sum / run_times))

Running Random Forest:   0%|                             | 0/10 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Running Random Forest:  10%|██                   | 1/10 [01:18<11:42, 78.02s/it]

Running not in develop mode
RMSE on test data: 7773.001991958654


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Running Random Forest:  20%|████▏                | 2/10 [02:35<10:20, 77.50s/it]

Running not in develop mode
RMSE on test data: 8690.393032576465


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Running Random Forest:  30%|██████▎              | 3/10 [03:52<09:01, 77.30s/it]

...

Running not in develop mode
RMSE on test data: 8286.823095767924


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Running Random Forest:  90%|██████████████████▉  | 9/10 [11:39<01:17, 77.70s/it]

Running not in develop mode
RMSE on test data: 12214.253363442933


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
Running Random Forest: 100%|████████████████████| 10/10 [12:56<00:00, 77.66s/it]

Running not in develop mode
RMSE on test data: 10462.550723668974
Average RMSE: 10040


### This cell do prediction model by model

In [293]:
x_train, y_train = df_aug.drop(columns=['price']), df_aug[['price', 'model']]
x_test = df_test[x_train.columns].dropna(subset=['model'])

res_model = RandomForestMiningByModel(x_train, x_test, y_train, dev=True)
print(res_model.head())

/Users/user/Documents/cs5228/CS5228_Project-mining_zijie/util/DataMining.py:120: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  y_pred = pd.concat([y_pred, temp_df])


0     19266.235
1     33973.670
2    143551.660
3     72275.440
4     27777.800
Name: Predicted, dtype: float64


### This cell do prediction on test data with 'model' attribute missing

In [294]:
df_test_unmapped = df_test[df_test['model'].isna()]
x_train, y_train = df_aug.drop(columns=['price', 'model']), df_aug[['price']]
x_test = df_test_unmapped[x_train.columns]

res_nomodel = RandomForestMining(x_train, x_test, y_train, dev=True)
print(res_nomodel.head())

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


         Predicted
21    60852.033916
195  285878.964933
212  187558.266407
402   20210.401628
412  103300.378208


In [295]:
print(len(res_model))
print(len(res_nomodel))
res = pd.concat([res_model, res_nomodel])
res.to_csv('./data/res_by_model_original.csv')
res.reset_index(inplace=True)
res.rename(columns={'index': 'Id'}, inplace=True)
res_sorted = res.sort_values(by='Id')
res_sorted.to_csv('./data/res_by_model2.csv', index=False)

9902
98
